In [31]:
# plot 라이브러리
import matplotlib
import matplotlib.pyplot as plt
# 경고 메시지 무시
import warnings
# 실행결과 경고메시지 출력 제외
warnings.filterwarnings('ignore')
# 글꼴 설정
plt.rc('font', family='NanumBarunGothic') 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.linear_model import SGDRegressor, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from pandas import read_csv
from pycaret.classification import setup
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
import os

#### 1. 반복학습 및 loss최저점 구하기, 조기 종료

In [6]:
np.random.seed(42)

m = 100
X = 6 * np.random.rand(m, 1) - 3

y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)

X_train, X_val, y_train, y_val = train_test_split(X, y.ravel(), test_size=0.5, random_state=10)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler()),
    ])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)

X_val_poly_scaled = poly_scaler.transform(X_val)

#warm start의 default는 False. 이는 fit을 실행할 때, 이전에 업데이터 된 weight를 초기화하고 다시 피팅한다는 것을 의미 
#반대로 True는 이전 호출에 사용했던 solution을 재사용 할지 여부를 결정
#eta0 = learning_rate 기본값 = 0.01 초기 학률
sgd_reg = SGDRegressor(max_iter=1,
                       penalty=None,
                       eta0=0.0005,
                       warm_start=True,
                       learning_rate="constant",
                       random_state=42)

train_errors, val_errors = [], []

n_epochs = 500

for epoch in range(n_epochs):
    sgd_reg.fit(X_train_poly_scaled, y_train)

    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)

    train_errors.append(mean_squared_error(y_train, y_train_predict))
    val_errors.append(mean_squared_error(y_val, y_val_predict))

best_epoch = np.argmin(val_errors)

best_val_rmse = np.sqrt(val_errors[best_epoch])


(50, 1) (50, 1) (50,) (50,)


In [7]:
iris = datasets.load_iris()
X = iris["data"][:, 3:]  
y = (iris["target"] == 2).astype(np.int)  

X_new = np.linspace(0, 3, 1000).reshape(-1, 1)

log_reg = LogisticRegression(solver='liblinear', random_state=42)
log_reg.fit(X, y)
y_proba = log_reg.predict_proba(X_new)

decision_boundary = X_new[y_proba[:, 1] >= 0.5][0]
decision_boundary

array([1.61561562])

In [8]:
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  
y = (iris["target"] == 2).astype(np.float64)  

svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C = 1, loss = 'hinge', random_state = 42))
])

svm_clf.fit(X, y)

svm_regression

In [ ]:
np.random.seed(42)
m = 50
X = 2 * np.random.rand(m, 1)
y = (4 + 3 * X + np.random.randn(m, 1)).ravel()


svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)

Decision_tree

In [ ]:
iris = load_iris()
X = iris.data[:, 2:]
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

In [ ]:
import graphviz
from sklearn import tree

dot_data = tree.export_graphviz(
        tree_clf,
        feature_names=["꽃잎 길이 (cm)", "꽃잎 너비 (cm)"],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

In [12]:
%cd /content/drive/MyDrive/kaggle/newdeal_class

/content/drive/MyDrive/kaggle/newdeal_class


In [16]:
titanic.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [18]:
titanic = pd.read_csv('./titanic.csv')

titanic.Sex = titanic.Sex.map({'female':0, 'male':1})
titanic.Age.fillna(titanic.Age.median(), inplace=True)

titanic.Embarked.fillna('S', inplace=True)
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked', drop_first=True)
titanic = pd.concat([titanic, embarked_dummies], axis=1)

feature_cols = ['Pclass', 'Sex', 'Age', 'Embarked_Q', 'Embarked_S']
X = titanic[feature_cols]
y = titanic.Survived

treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})

,feature,importance
0,Pclass,0.242664
1,Sex,0.655584
2,Age,0.064494
3,Embarked_Q,0.000000
4,Embarked_S,0.037258


In [21]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

log_clf = LogisticRegression(solver='liblinear', random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma='auto', random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(random_state=42,
                                                 solver='liblinear')),
                             ('rf',
                              RandomForestClassifier(n_estimators=10,
                                                     random_state=42)),
                             ('svc', SVC(gamma='auto', random_state=42))])

In [ ]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

In [23]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=0.1, random_state=42)
gbrt.fit(X, y)

GradientBoostingRegressor(max_depth=2, n_estimators=3, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,mean_squared_error

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
model = GradientBoostingClassifier(random_state=1) 
param_test = { "n_estimators": range(50, 100, 25),
 "max_depth": [1, 2, 4],
  "learning_rate": [0.0001, 0.001, 0.01, 0.1],
  } 
  
Gsearch = GridSearchCV(model, param_grid=param_test, scoring="accuracy", n_jobs=-1, cv=5 )
Gsearch.fit(X_train,y_train)
print(Gsearch.best_params_)

In [ ]:
import joblib
joblib.dump(Gsearch, './model.pkl')
loaded_model = joblib.load('./model.pkl')
loaded_model.predict(X_test)

In [ ]:
import xgboost
xgb_reg = xgboost.XGBRegressor(random_state=42)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)
val_error = mean_squared_error(y_val, y_pred)
print("검증 MSE:", val_error)

xgb_reg.fit(X_train, y_train,eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)
val_error = mean_squared_error(y_val, y_pred)
print("검증 MSE:", val_error)

Stacking

In [ ]:
from sklearn.ensemble import  StackingClassifier
from sklearn.naive_bayes import GaussianNB

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

base_models = [
                 ('rf_1', RandomForestClassifier(max_depth=10)),
                 ('rf_2', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', max_depth=1, random_state=1),
                   learning_rate=0.1, n_estimators=1000)) 
                ]

clf = StackingClassifier(estimators=base_models, final_estimator=GradientBoostingClassifier(max_depth=4, n_estimators=75, random_state=1))

clf.fit(X_train,y_train)

In [25]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.7 MB/s eta 0:00:00
 

In [27]:
from pycaret.datasets import get_data
data = get_data('juice')
data.info()

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              1070 non-null   int64  
 1   Purchase        1070 non-null   object 
 2   WeekofPurchase  1070 non-null   int64  
 3   StoreID         1070 non-null   int64  
 4   PriceCH         1070 non-null   float64
 5   PriceMM         1070 non-null   float64
 6   DiscCH          1070 non-null   float64
 7   DiscMM          1070 non-null   float64
 8   SpecialCH       1070 non-null   int64  
 9   SpecialMM       1070 non-null   int64  
 10  LoyalCH         1070 non-null   float64
 11  SalePriceMM     1070 non-null   float64
 12  SalePriceCH     1070 non-null   float64
 13  PriceDiff       1070 non-null   float64
 14  Store7          1070 non-null   object 
 15  PctDiscMM       1070 non-null   float64
 16  PctDiscCH       1070 non-null   float64
 17  ListPriceDiff   1070 non-null   f

In [28]:
from pycaret.datasets import get_data 
from pycaret.classification import *
setup_clf = setup(data=data, target='Purchase')

,Description,Value
0,Session id,746
1,Target,Purchase
2,Target type,Binary
3,Target mapping,"CH: 0, MM: 1"
4,Original data shape,"(1070, 19)"
5,Transformed data shape,"(1070, 19)"
6,Transformed train set shape,"(749, 19)"
7,Transformed test set shape,"(321, 19)"
8,Ordinal features,1
9,Numeric features,17


In [29]:
top = compare_models()
top

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8264,0.8998,0.7393,0.8025,0.7680,0.6297,0.6328,0.5100
lda,Linear Discriminant Analysis,0.8224,0.8915,0.7361,0.7967,0.7640,0.6220,0.6244,0.3180
ridge,Ridge Classifier,0.8198,0.0000,0.7329,0.7943,0.7609,0.6167,0.6194,0.1180
lr,Logistic Regression,0.8158,0.8944,0.7191,0.7929,0.7527,0.6065,0.6098,1.6050
lightgbm,Light Gradient Boosting Machine,0.8131,0.8778,0.7432,0.7740,0.7559,0.6048,0.6076,0.4630
ada,Ada Boost Classifier,0.8130,0.8703,0.7157,0.7898,0.7491,0.6008,0.6044,0.3740
xgboost,Extreme Gradient Boosting,0.8091,0.8741,0.7431,0.7659,0.7509,0.5965,0.6003,0.3060
rf,Random Forest Classifier,0.7917,0.8764,0.7056,0.7516,0.7261,0.5584,0.5608,0.9420
et,Extra Trees Classifier,0.7770,0.8448,0.6920,0.7313,0.7096,0.5288,0.5307,0.6500
knn,K Neighbors Classifier,0.7610,0.8163,0.6402,0.7172,0.6746,0.4871,0.4906,0.3070


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=746, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [30]:
from pycaret.regression import *
from pycaret.datasets import get_data
dataset = get_data('diamond')
exp = setup(dataset, target='Price')
reg_top = compare_models()
reg_top

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


,Description,Value
0,Session id,2671
1,Target,Price
2,Target type,Regression
3,Original data shape,"(6000, 8)"
4,Transformed data shape,"(6000, 29)"
5,Transformed train set shape,"(4200, 29)"
6,Transformed test set shape,"(1800, 29)"
7,Ordinal features,1
8,Numeric features,1
9,Categorical features,6


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,687.5073,2202554.0064,1442.0372,0.9798,0.0717,0.0533,0.8720
et,Extra Trees Regressor,745.8728,2498276.3681,1540.8831,0.9772,0.0791,0.0588,2.0210
rf,Random Forest Regressor,760.7219,3022891.6436,1689.0092,0.9722,0.0794,0.0582,1.8030
lightgbm,Light Gradient Boosting Machine,749.2417,3259762.9175,1752.8946,0.9703,0.0775,0.0563,0.6480
gbr,Gradient Boosting Regressor,934.7073,3691819.6529,1898.5399,0.9657,0.1017,0.0771,0.9280
dt,Decision Tree Regressor,998.6870,5572139.6293,2304.4224,0.9490,0.1059,0.0758,0.4790
lasso,Lasso Regression,2523.8688,14921792.0025,3835.2296,0.8621,0.6252,0.2985,0.5680
llar,Lasso Least Angle Regression,2523.8892,14921782.4445,3835.2293,0.8621,0.6252,0.2985,0.2370
br,Bayesian Ridge,2527.9947,14937096.3466,3837.6514,0.8619,0.6222,0.2993,0.3660
lr,Linear Regression,2536.3725,14975302.8760,3842.4413,0.8616,0.6492,0.3003,0.3380


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=None,
             objective='reg:squarederror', predictor=None, ...)

In [32]:
# pycaret classification 모델들을 비교하기 위한 모듈
from pycaret.classification import compare_models

# sonar 데이터셋
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv'

# 데이터셋 로드
df = read_csv(url, header=None)
# 컬럼 갯수
n_cols = df.shape[1]

# 컬럼명 String으로 반환
df.columns = [str(i) for i in range(n_cols)]
# 데이터셋 설정
grid = setup(data=df, target=df.columns[-1],  verbose=True)
# 모델 학습
best = compare_models()
# best 모델 확인
print(best)

,Description,Value
0,Session id,5941
1,Target,60
2,Target type,Binary
3,Target mapping,"M: 0, R: 1"
4,Original data shape,"(208, 61)"
5,Transformed data shape,"(208, 61)"
6,Transformed train set shape,"(145, 61)"
7,Transformed test set shape,"(63, 61)"
8,Numeric features,60
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8276,0.9537,0.7690,0.8588,0.8024,0.6518,0.6644,0.6500
lightgbm,Light Gradient Boosting Machine,0.8271,0.9077,0.7810,0.8502,0.8068,0.6513,0.6613,0.1440
gbc,Gradient Boosting Classifier,0.8138,0.8864,0.7690,0.8411,0.7953,0.6253,0.6360,0.5130
xgboost,Extreme Gradient Boosting,0.8138,0.8898,0.7833,0.8133,0.7951,0.6252,0.6288,0.2520
rf,Random Forest Classifier,0.8000,0.8956,0.7238,0.8398,0.7641,0.5946,0.6132,0.8280
ada,Ada Boost Classifier,0.7995,0.8540,0.7524,0.8130,0.7759,0.5963,0.6038,0.3730
ridge,Ridge Classifier,0.7667,0.0000,0.7548,0.7636,0.7492,0.5332,0.5461,0.1000
knn,K Neighbors Classifier,0.7652,0.8434,0.6667,0.8292,0.6897,0.5219,0.5615,0.1120
lr,Logistic Regression,0.7581,0.8400,0.7095,0.7889,0.7320,0.5151,0.5322,0.3410
lda,Linear Discriminant Analysis,0.7448,0.8600,0.7357,0.7275,0.7241,0.4869,0.4955,0.2130


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=5941, verbose=0, warm_start=False)
